In [1]:
#Used open-source API to grab flight info from QPX Express
#google_flight from https://github.com/yangxu/google_flight_api

import json
from google_flight import google_flight_api

def price_grab(origin, dest):
    "Grabs price of cheapest flight to given destination airport from given origin airport"
    g = google_flight_api.GoogleFlight("***API KEY***")
    data = {             #request to QPX Express for flight info
          "request": {
            "slice": [
              {
                "origin": origin,
                "destination": dest,
                "date": "2017-12-28",
                "maxStops": 0,
                "permittedDepartureTime": {
                  "earliestTime": "00:00",
                  "latestTime": "23:59"
                }
              }
            ],
            "passengers": {
              "adultCount": 1,
              "infantInLapCount": 0,
              "infantInSeatCount": 0,
              "childCount": 0,
              "seniorCount": 0
            },
            "solutions": 5,
            "refundable": False
          }
    
        }
    #attempt to get flight info
    try: 
        g.get(data)
    except:
        return 10000.0 #if no flights, return $10,000
    json_data = g.trips
    return float(json_data[0]['pricing'][0]['saleTotal'][3:]) #if flight found, cut off currency info and return price as float

In [2]:
#initialize flight and airport data, set up Bokeh server

import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show

output_notebook()

flights = pd.read_csv('flights_df.csv')
flights = flights.iloc[:,2:]
airports = pd.read_csv('airports.csv')
flights.head()

Loading BokehJS ...

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,ARRIVAL_DELAY,ORIG_CITY,ORIG_STATE,ORIG_LATITUDE,ORIG_LONGITUDE,DEST_CITY,DEST_STATE,DEST_LATITUDE,DEST_LONGITUDE
0,ANC,SEA,-22.0,Anchorage,AK,61.17432,-149.99619,Seattle,WA,47.44898,-122.30931
1,ANC,SEA,-14.0,Anchorage,AK,61.17432,-149.99619,Seattle,WA,47.44898,-122.30931
2,ANC,SEA,-24.0,Anchorage,AK,61.17432,-149.99619,Seattle,WA,47.44898,-122.30931
3,ANC,SEA,NaN,Anchorage,AK,61.17432,-149.99619,Seattle,WA,47.44898,-122.30931
4,ANC,SEA,-35.0,Anchorage,AK,61.17432,-149.99619,Seattle,WA,47.44898,-122.30931


In [6]:
def best_flights(org_airport):
    "Given origin airport, creates a table of top 3 destinations based on delay percent and cost of flight."
    dct = {}
    origin = flights[flights.ORIGIN_AIRPORT == org_airport] #limit data to flights from given airport
    destinations = origin.DESTINATION_AIRPORT.unique() #generate list of potential destinations
    for index, dest in enumerate(destinations):
        if len(origin[origin.DESTINATION_AIRPORT == dest].ARRIVAL_DELAY) > 300: #ensure there are enough flights to destination for accurate data
            delay_pct = len(origin[(origin.DESTINATION_AIRPORT == dest) & (origin.ARRIVAL_DELAY >= 15)].ARRIVAL_DELAY)/ len(origin[origin.DESTINATION_AIRPORT == dest].ARRIVAL_DELAY)
            dct[index] = (dest, delay_pct)
    dct_df = pd.DataFrame.from_dict(dct, orient='index')
    dct_df.columns = ['dest', 'delay_pct']
    top_ten = dct_df.sort_values('delay_pct').head(10).reset_index(drop=True) #restrict new table to only top ten to not waste money on QPX Exress queries
    #initialize new columns with max values
    top_ten['cost'] = 10000.0 
    top_ten['orank'] = 10000.0
    #loop over destinations to grab price info
    for i in range(len(top_ten.dest)):
        cost = price_grab(org_airport, top_ten.dest[i])
        rank = cost * top_ten.delay_pct[i]
        top_ten.loc[i, 'cost'] = '$' + str(cost) + '0'
        top_ten.loc[i, 'orank'] = rank
    #sort by orank (original rank value), then create column for 1-3 ranking to show to user
    top_ten = top_ten.sort_values('orank')
    top_ten.index = range(1,len(top_ten.dest)+1) #set index to match size, some airports may have <10 destinations
    top_three = top_ten.reset_index().head(3)
    top_three.rename(columns={'index':'rank'}, inplace=True)
    top_three['orig'] = org_airport
    lat = float(airports[airports['IATA_CODE'] == org_airport].LATITUDE.values)
    lon = float(airports[airports['IATA_CODE'] == org_airport].LONGITUDE.values)
    top_three['orig_lat'] = lat
    top_three['orig_lon'] = lon
    #merge with airport df to get latitude and longitude info for graph
    top_merge = top_three.merge(airports, how='left', left_on='dest', right_on='IATA_CODE', copy=False)
    return top_merge

In [7]:
top = best_flights("LGA")
top

In [11]:
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, Range1d, HoverTool
from bokeh.io import push_notebook

from ipywidgets import interact

orig_airport = 'LGA'

usmap = "USMAP.PNG"
map_src = ColumnDataSource(dict(url = [usmap]))

graph = figure(plot_width = 799, plot_height = 572, title="")
graph.toolbar.logo = None
graph.toolbar_location = None
graph.x_range=Range1d(start=-180, end=-50)
graph.y_range=Range1d(start=20, end=80)
graph.image_url(url='url', x=-180, y = 80, w=130, h=60, source=map_src)
graph.outline_line_alpha = 0 
graph.title.text = (orig_airport)
#curdoc().add_root(graph)

source = ColumnDataSource(top)
o_source =  ColumnDataSource(dict(
        orig = flights[flights.ORIGIN_AIRPORT == orig_airport].ORIGIN_AIRPORT.head(1),
        orig_lat = airports[airports.IATA_CODE == orig_airport].LATITUDE,
        orig_lon = airports[airports.IATA_CODE == orig_airport].LONGITUDE 
    ))

def update(orig_airport):
    top = best_flights(orig_airport)
    #destination data
    new_data = dict(
        orig_lon = top.orig_lon,
        orig_lat = top.orig_lat,
        LONGITUDE = top.LONGITUDE,
        LATITUDE = top.LATITUDE,
        dest = top.dest,
        orig = top.orig,
        city = top.CITY,
        state = top.STATE,
        rank = top['rank']
    )
    #origin data
    new_other_data = dict(
        orig = flights[flights.ORIGIN_AIRPORT == orig_airport].ORIGIN_AIRPORT.head(1),
        orig_lat = airports[airports.IATA_CODE == orig_airport].LATITUDE,
        orig_lon = airports[airports.IATA_CODE == orig_airport].LONGITUDE 
    )
    #reset sources to new data and update title
    source.data = new_data
    o_source.data = new_other_data
    graph.title.text = (orig_airport)
    push_notebook()

    
origin = graph.diamond(x='orig_lon', y='orig_lat', size=9, color='red', alpha=1, source=o_source)
graph.add_tools(HoverTool(renderers=[origin], tooltips=[('Origin','@orig')]))

ap_source = ColumnDataSource(top)

destinations = graph.circle(x='LONGITUDE', y='LATITUDE', size=7, color='black', alpha=.7, source=source)
graph.add_tools(HoverTool(renderers=[destinations], tooltips=[('Destination','@dest'), ('City', '@city'), ('State','@state'), ('Rank', '@rank')]))

interact(update, orig_airport = flights.ORIGIN_AIRPORT.unique())
show(graph, notebook_handle=True)
